To dos
1) Set up script to first initialize database image collection (create image collection and write ID image) and then to loop over the dates in the input image collection
2) Add additional reducers  
- For continuous datasets - 5th, 25th, 50th, 75th, and 95th percentile, mean 
- For categorical datasets - Histogram
2) Set up to handle reducers continuous and/or categorical datasets 
3) Add checks whether image asset already exists  
4) Handle NAs
5) Set export parameters as properties (resolution, continuous vs. categorical, unit reduced)  
6) Right now the exports are run over a specified period of time (one month, one year, etc. — iterated), but may want to assess applying to one image at a time
7) Test with RAP

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh6AIKjDjeVn9Zy5YuIG6E9cczESz9_4ceQDqNChGAUhPtky9wKwOn0

Successfully saved authorization token.


In [91]:
import os
import datetime
from datetime import date
from datetime import timedelta
import pandas as pd

In [92]:
# !pip install geemap
import ee
import geemap
Map = geemap.Map()

## Load datasets

In [130]:
# ------------------------------------- Define parameters -----------------------------------------------

# Define time period to export
daysToExport = 30
startDate = datetime.datetime(2022, 1, 1)

# Define whether to initialize new image collection or append to existing image collection
process = 'initialize' 


# -------------------------------- Define input Image Collection ----------------------------------------

# Define input Image Collection
in_ic = ee.ImageCollection("GRIDMET/DROUGHT").filterDate(startDate, startDate + timedelta(days = daysToExport))

# Define variable to generate database table for
var_name = 'long_term_blend'
in_var_type = 'continuous'

# Other datasets
# gm_blends = ee.ImageCollection("GRIDMET/DROUGHT")
# rap_npp = ee.ImageCollection("projects/rangeland-analysis-platform/npp-partitioned-v3")
# rap_cov = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
# gm_mat = ee.ImageCollection("projects/rangeland-analysis-platform/gridmet-MAT")


# ------------------------------- Define input Feature Collection ---------------------------------------

# in_fc = ee.FeatureCollection('projects/dri-apps/assets/blm-admin/BLM_Natl_Grazing_Allotment_Polygons')
in_fc = ee.FeatureCollection('projects/dri-apps/assets/BLM_Natl_Grazing_Allotment_Polygons_Simplified_clean')

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-108.4020, 38.7855], [-108.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 38.7855]]], None, False);
# in_fc = in_fc.filterBounds(geometry)
# # Specify ID property
# in_fc_id = "ALLOT_ID"

# Use full Feature Collection
in_fc = in_fc
# Specify ID property
in_fc_id = "ALLOT_ID"

## Pre-process data

### Calculate GridMET drought blends

In [131]:
# Define property list
property_list = ["system:index", "system:time_start"]

# Function to calculate short-term and long-term blends
def preprocess_gm_drought(img):

  # Define preliminary variables for short-term blend calculation
  stb_variable = "short_term_blend"
  stb_pdsi_img = img.select("pdsi")
  stb_z_img = img.select("z")
  stb_spi90d_img = img.select("spi90d")
  stb_spi30d_img = img.select("spi30d")

  # Define weights for short-term blend calculation
  stb_pdsi_coef = 0.2
  stb_z_coef = 0.35
  stb_spi90d_coef = 0.25
  stb_spi30d_coef = 0.2

  # Calculate short-term blend
  stblend = stb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi90d * spi90d_coef + spi30d * spi30d_coef + z * z_coef / 2",{
          "spi90d": stb_spi90d_img, 
          "spi30d": stb_spi30d_img, 
          "z": stb_z_img, 
          "pdsi_coef": stb_pdsi_coef,
          "spi90d_coef": stb_spi90d_coef, 
          "spi30d_coef": stb_spi30d_coef, 
          "z_coef": stb_z_coef})

  # Define preliminary variables for long-term blend calculation
  ltb_variable = "long_term_blend"
  ltb_pdsi_img = img.select("pdsi")
  ltb_spi180d_img = img.select("spi180d")
  ltb_spi1y_img = img.select("spi1y")
  ltb_spi2y_img = img.select("spi2y")
  ltb_spi5y_img = img.select("spi5y")

  # Define weights for long-term blend calculation
  ltb_pdsi_coef = 0.35
  ltb_spi180d_coef = 0.15
  ltb_spi1y_coef = 0.2
  ltb_spi2y_coef = 0.2
  ltb_spi5y_coef = 0.1

  # Calculate short-term blend
  ltblend = ltb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi180d* spi180d_coef + spi1y * spi1y_coef + spi2y * spi2y_coef + spi5y * spi5y_coef",{
          "spi180d": ltb_spi180d_img, 
          "spi1y": ltb_spi1y_img, 
          "spi2y": ltb_spi2y_img, 
          "spi5y": ltb_spi5y_img,
          "spi180d_coef": ltb_spi180d_coef, 
          "spi1y_coef": ltb_spi1y_coef, 
          "spi2y_coef": ltb_spi2y_coef,
          "spi5y_coef": ltb_spi5y_coef, 
          "pdsi_coef": ltb_pdsi_coef})
  
  return ltblend.addBands(stblend).select([0,1], [ltb_variable, stb_variable]).copyProperties(img, property_list)

# Map function to calculate drought blends over the subset years, convert to monthly median images, and convert to multi-band image
# Filter for dates without NAs for the long term blend
in_ic = in_ic.filterDate('1985-01-01', str(date.today()))\
                 .map(preprocess_gm_drought)

# Convert Image Collection to multi-band image
in_i = in_ic.toBands()

# Select variable to serve as input
in_i = in_i.select(['[0-9]{8}_' + var_name])

# Bandnames must be an eight digit character string 'YYYYMMDD'. Annual data will be 'YYYY0101'.
def replace_name(name):
  return ee.String(name).replace(var_name, '').replace('_', '')

# Finish cleaning input image
in_i = in_i.rename(in_i.bandNames().map(replace_name))
print(in_i.bandNames().getInfo())

['20220105', '20220110', '20220115', '20220120', '20220125', '20220130']


## Reduce regions to time-series centroid Feature Collection

In [114]:
# Function to return feature time-series as centroid feature collection for continuous variabless
def imgs_to_pts_continuous(img):
    
    # Cast input image to ee.Image
    img = ee.Image(img)
    
    # Get resolution of the image
    res = img.select(0).projection().nominalScale()
    
    # Run reduce regions for allotments
    img_rr = img.reduceRegions(collection = in_fc,\
                                 reducer = ee.Reducer.percentile([5, 25, 50, 75, 95]),\
                                 scale = res,\
                                 tileScale = 16)
    
    # Function to clean up property names
    def clean_f(f):
        
        # Get ID for the input feature
        f_id = f.get(in_fc_id)
        
        # Rename ID for later sorting
        f = f.set("0_id", ee.Number.parse(f_id))
        
        # Select ID band and time-series bands and convert to centroids
        return(ee.Feature(f.select(['0_id', '[0-9]{8}.*']))\
                       .centroid())
    
    # Apply function to clean up property names
    img_rr = img_rr.map(clean_f)
    
    # Get list of RR features
    img_rr_list = img_rr.toList(img_rr.size())
    
    # Get size of RR features
    img_rr_size = img_rr_list.size()
    
    # Function to create feature collection next to equator
    def pts_to_equator(i):
        
        # Cast number to EE number
        i = ee.Number(i)
        
        # Get properties
        properties = ee.Feature(img_rr_list.get(i)).toDictionary()
        
        # Create geometry at equator
        geom = ee.Geometry.Point([i.multiply(0.0002), 0.0002])
        
        # Return object with properties
        return(ee.Feature(geom).set(properties))
    
    # Create equator feature collection
    proxy_fc = ee.FeatureCollection(ee.List.sequence(0, img_rr_size.subtract(1), 1).map(pts_to_equator))
    
    return(proxy_fc)

In [115]:
# Run function to get time-series statistics for input feature collection for continuous variables
rr_fc = imgs_to_pts_continuous(in_i)
print(rr_fc.first().propertyNames().sort().getInfo())
print(rr_fc.size().getInfo())

['0_id', '20220105_p25', '20220105_p5', '20220105_p50', '20220105_p75', '20220105_p95', '20220110_p25', '20220110_p5', '20220110_p50', '20220110_p75', '20220110_p95', '20220115_p25', '20220115_p5', '20220115_p50', '20220115_p75', '20220115_p95', 'system:index']
21591


## Convert centroid time-series to Image Collection time-series

In [116]:
# Function to generate series image collection from feature collections
def pts_to_ics_continuous(in_fc):

    # Get list of property names
    def slice_strs(str):
        return(ee.String(str).slice(0,8))

    # Get list of unique dates   
    fc_dates = in_fc.first().propertyNames().sort().remove("system:index").map(slice_strs).distinct() 

    # Generate time-series feature collection
    def generate_date_fc(date):
        date = ee.String(date)

        # Select the properties for the date
        def filter_f_properties(f):
            # Select properties for the date
            f_sub = ee.Feature(f).select([date.cat('.*')])

            # Function to remove date from property names
            def clean_property_names(prop):

                return(ee.String(prop).replace(date.cat('_'), '').replace('0_id', 'id'))

            # Remove dates from property names
            f_sub = ee.Feature(f_sub).select(f_sub.propertyNames(), f_sub.propertyNames().map(clean_property_names))
            return(ee.Feature(f_sub))

        # Apply function to select the Feature properties for the date and add new properties for the output Feature Collection
        f_sub_clean = ee.FeatureCollection(in_fc.map(filter_f_properties)).set('system:index', date).set('f_id', date)

        return(f_sub_clean)

    # Apply function to produce time-series feature collection
    fc_dates = fc_dates.map(generate_date_fc)

    # Convert dates Feature Collection to Image Collection
    def fcs_to_ic(fc):
        # Cast to FeatureCollections
        fc = ee.FeatureCollection(fc)

        # Get list of properties to iterate over for creating multiband image for each date
        props = fc.first().propertyNames().remove('system:index')

        # Function to generate image from stats stored in Feature Collection property
        def generate_stat_image(prop):
            img = fc.reduceToImage(properties = [prop], reducer = ee.Reducer.sum()).rename([prop])
            return(img)

        # Generate multi-band stats image
        img_mb = ee.ImageCollection(props.map(generate_stat_image)).toBands().rename(props).set("system:index", fc.get('f_id')).set("var_name", var_name)

        return(img_mb)
    
    ic_from_fcs = fc_dates.map(fcs_to_ic)
    
    return(ic_from_fcs)

In [117]:
# Convert centroid time-series to image collection time-series
rr_ic = ee.ImageCollection(pts_to_ics_continuous(rr_fc))
# print(ee.Image(rr_ic.toList(10).get(1)).bandNames().getInfo())

In [31]:
# # Create export geometry
# geometry = rr_fc.geometry().bounds().buffer(1000)

# Map = geemap.Map()
# Map.addLayer(ee.Image(rr_ic.first()))
# Map.addLayer(geometry)
# Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Export Images to Image Collection Asset

In [118]:
# -------- Define export parameters -----------
# Define asset to export
out_ic = rr_ic
out_ic_str = 'gridmet-' + var_name.replace('_', '-')
print(out_ic_str)

# Parse Image Collection ID
ic_id = f"projects/dri-apps/assets/blm-database/{out_ic_str}"

# Generate empty Image Collection asset to append images
os.system(f"earthengine create collection {ic_id}")

# -------- Get list of image IDs -----------
# Get image ID
# Return from EE IC
img_ids = out_ic.aggregate_array('system:index').getInfo()
# print(img_ids)

# -------- Create export tasks ---------
# Create export geometry
geometry = rr_fc.geometry().bounds().buffer(300)

# Loop over IDs to generate export tasks
for i, img_id in enumerate(img_ids):
    img = ee.Image(out_ic.filter(ee.Filter.eq('system:index', img_id)).first())
    task = ee.batch.Export.image.toAsset(
        image = ee.Image(img),
        description = img_id,
        assetId = ic_id + "/" + img_id,
        scale = 22.264,
        region = geometry,
        maxPixels = 1e13
    )
    
    task.start()
    print(img_id)

gridmet-long-term-blend
Asset projects/dri-apps/assets/blm-database/gridmet-long-term-blend already exists.


In [76]:
!earthengine rm --recursive projects/ce-datasets/assets/blm-allotment-ics/blm-rap-cover-afg